<a href="https://colab.research.google.com/github/Hamtori82/low_fertility_policy/blob/master/%EB%84%A4%EC%9D%B4%EB%B2%84%EB%89%B4%EC%8A%A4%EB%8C%93%EA%B8%80%ED%81%AC%EB%A1%A4%EB%A7%81_%EC%A0%80%EC%B6%9C%EC%82%B0%EA%B7%B9%EB%B3%B5%EC%A0%95%EC%B1%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Colab에 Mecab 설치
# 코랩에서 한글 표기를 위해 폰트 설치

In [5]:
flag = True
while flag :
  !git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
  %cd Mecab-ko-for-Google-Colab
  !bash install_mecab-ko_on_colab190912.sh

  !apt-get update -qq
  !apt-get install fonts-nanum* -qq  
  flag = False

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 9.86 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 465 kB 52.1 MB/s 
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-11-16 00:11:57--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::3403:4be7, 2406:da00:ff00::22c0:3470, ...
Connecting to bitbucket.org (bi

In [ ]:
### 라이브러리 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from collections import Counter
from konlpy.tag import Mecab
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 키워드 입력 후 크롤링

In [2]:
from bs4 import BeautifulSoup
import requests
import re
import sys
import pprint
import csv
import json
import pandas as pd

result=[]

headers = {
    'authority': 'apis.naver.com',
    'accept': '*/*',
    'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    # Requests sorts cookies= alphabetically
    # 'cookie': 'ab.storage.userId.7af503ae-0c84-478f-98b0-ecfff5d67750=%7B%22g%22%3A%22browser-1625985144309-6%22%2C%22c%22%3A1626101500089%2C%22l%22%3A1626101500089%7D; ab.storage.deviceId.7af503ae-0c84-478f-98b0-ecfff5d67750=%7B%22g%22%3A%224cbe130c-6edd-d4aa-a78d-290b003c3592%22%2C%22c%22%3A1626101500094%2C%22l%22%3A1626101500094%7D; ASID=7992e0220000017aaa36664e0000004e; _ga=GA1.2.612969395.1626832328; ab.storage.sessionId.7af503ae-0c84-478f-98b0-ecfff5d67750=%7B%22g%22%3A%2228148006-e01d-7623-b7d1-b4fff0f59b4e%22%2C%22e%22%3A1627919390179%2C%22c%22%3A1627908091281%2C%22l%22%3A1627917590179%7D; MM_NEW=1; NFS=2; NNB=RDIIILNX6JCWE; nx_ssl=2; nid_inf=1665554565; NID_AUT=tP3V5ox533EjyAgkJ1JaqWEnPOhXs2hr3teD39pK972fuXqDWQZXoIOMzICJpa1A; NID_JKL=d393brIzilbjw+7TVvG0OW6Eo22+WIhQAfihItUdgbY=; _naver_usersession_=SPdJTrlTMrn8Udkyn58eo6HL; NID_SES=AAABwJaKJ5FjUAETXL8SAX2HKMUSTt3l8pPu49OSzbGzgKEEMN/ckpP4DbQVHQwTV1hVPWtbpP7Nomg0CbD8TtCpyOYbeq8+OpHb5eWbDsXXCeLHO4epgthLtbQHiBE8spXqEtx/h0D6MzxsIlN4pa8gz51jV+oWzQQNnpQCeaKKLaxcpMfhGXnZv4BK1Rg+TAgUFE9RtExcKjteTL2hB9tKT41C7antdQdhLfVXWUbsJ/q5b62iDZnnZUAANXHnWp/9RI2YyKSn70SVu4Bag+fxA/23OqjCHSbK5RMiNOQKV+Bs7uugaAsMKkH6lGBBIbNDkTXGZ4n1+KbqFwe1kV9oCaPJ+siwXESEqvY0jaLVNAqUATQZjnIMFIYwARw41FTuduxW1IOF7MdP7R3EqOvnqNir2lXW1UfRlHlOtMC4w/tXk8xqJR/HVlZrnltKkMZB5zfyDNvnt02jbOKJcORjmOeVvL+xoCdSXwZclfJzRkC31l43+9jSu4X8RPUfuJILRMHf2e1A0NU7Mwds7h+S//5AD0yUJlPtFFzLvriuD1SMTRXiSwN4pNWBi6UIsPzScRpyLMc8hUE8Bi8jJtGk4e0=; NDARK=N; page_uid=hrKUflprvN8ssNc4Muwssssss3R-382317; BMR=',
    'referer': 'https://n.news.naver.com/article/028/0002595736',
    'sec-ch-ua': '"Whale";v="3", " Not;A Brand";v="99", "Chromium";v="102"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'script',
    'sec-fetch-mode': 'no-cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.108 Whale/3.15.136.18 Safari/537.36',
}

def news_url(query):
    page =1 
    url_list=[]
    while page <= 151 :

        ## 2022년 기사만 (2022.01.01 ~ 10.17)
        url = "https://m.search.naver.com/search.naver?where=m_news&sm=mtb_pge&query="+query+"&sort=0&photo=0&field=0&pd=3&ds=2022.01.01&de=2022.10.17&cluster_rank=24&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from20220101to20221017&start="+str(page)   

        #url = "https://m.search.naver.com/search.naver?where=m_news&sm=tab_pge&query="+query+"&sort=0&photo=0&field=0&pd=1&ds=&de=&cluster_rank=129&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:1w,a:all&start="+str(page)
        response = requests.get(url,headers=headers)
        html = response.text

        #뷰티풀소프의 인자값 지정
        soup = BeautifulSoup(html, "lxml")
        # print(soup.select('a.news_tit')[3]['href'])
        #<a>태그에서 제목과 링크주소 추출
        atags = soup.select('.news_tit')
        #print(len(atags))
        for i in atags:
            if "https://n.news.naver.com/" in i['href'] :
                url_list.append(i['href'])   #링크주소
                #print(i['href'])
        if len(atags) < 15 : #한페이지에 15개나오는데, 그보다 작으면 없다고 판단
            break
        page = page + 15
    return url_list
    # 위에서 크롤링한 네이버 뉴스 url을 입력합니다.

def comment(url_list):
    total_comment = [] #모든 댓글을 담는 리스트
    for url_ex in url_list:
        url = url_ex
        url = url.split('?')[0]
        oid_1 = url.split('/')[-1]
        oid_2 = url.split('/')[-2]
        i = 1
        # df = pd.DataFrame(columns=['댓글'])

        while True:
            params = {
                'ticket': 'news',
                'templateId': 'default_society',
                'pool': 'cbox5',
                'lang': 'ko',
                'country': 'KR',
                'objectId': f'news{oid_2},{oid_1}',
                'pageSize': '100',
                'indexSize': '10',
                'page': str(i),
                'currentPage': '0',
                'moreParam.direction': 'next',
                'moreParam.prev': '10000o90000op06guicil48ars',
                'moreParam.next': '1000050000305guog893h1re',
                'followSize': '100',
                'includeAllStatus': 'true',
            }

            response = requests.get('https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json', params=params, headers=headers)
            response.encoding = "UTF-8-sig"
            res = response.text.replace("_callback(","")[:-2]
            temp=json.loads(res) 
            # print(temp)
            try : 
                comment = list(pd.DataFrame(temp['result']['commentList'])['contents'])
                for j in range(len(comment)):
                    total_comment.append(comment[j])
                if len(comment)<97: 
                    break 
                else: 
                    i+=1
            except :
                break    

    print(total_comment)
    return total_comment



In [12]:
# 크롤링 실행 함수
def main(query):
    url_list = news_url(query)  
    allCommetns = comment(url_list)

    ## 리스트 -> 데이터프레임
    cols_name = ["reply"]
    total_data = pd.DataFrame(allCommetns,columns=cols_name)

    #print(len(total_data))
    total_data[:5]

    ## 크롤링 결과 저장
    total_data.to_csv(f'/content/{query}.csv',encoding='utf-8')



In [14]:
query = ['영아수당 지급', '첫만남 꾸러미 도입', '부부육아 휴직 활성화', '공보육 확충', '다자녀 지원 확대']
for i in query:
    main(i)

['무슨 중복이요? 그럼 휴직급여 그거 하나 받고 키우라는건디 다 일 그만두라는거에요 ? 지원금이 평생 나오는것도 아니고 영유아기 잠시 나오는건데도 이 난리를쳐서 못받게하네', '세금 내고 일하는 워킹맘이 죄인인가? 부모급여 줄 것처럼 언플해서 원래 지자체에서 자체적으로 주던 지원금까지 끊겼다. 육아휴직 수당 받아도 원래 받던 월급에 한참 못 미치는데 여기에 부모급여까지 안 주면 신생아 맡기고 빨리 복직하라는 소리 밖에 더 됨? 육아휴직 수당 상한선을 높이던지. 신생아라서 필수적으로 엄마들이 휴직해야 하는 시기에 경제적으로 지원도 안해줄 거면서 무슨 애를 낳으라고.', '와 1년 주는거도 건드리네 더줄생각은 못하고', '통상임금 80프로 해봐야 물가대비 애키우는돈에 턱없이부족한데 저거 70받을라고 일다그만두라고?', '제정신인가 OOO들이 그럼 다실업자되고 70만원받고땐치라는건가?', '무슨 중복인지? 그럼 다 경력단절되어야하는건가?', '기사놈 사회성 없는거같음', '국가에서 키워서나눠주자 .지자식을왜국가에서 세금퍼주는건데', '세금 내고 일하는 워킹맘이 죄인인가? 부모급여 줄 것처럼 언플해서 원래 지자체에서 자체적으로 주던 지원금까지 끊겼다. 육아휴직 수당 받아도 원래 받던 월급에 한참 못 미치는데 여기에 부모급여까지 안 주면 신생아 맡기고 빨리 복직하라는 소리 밖에 더 됨? 육아휴직 수당 상한선을 높이던지. 신생아라서 필수적으로 엄마들이 휴직해야 하는 시기에 경제적으로 지원도 안해줄 거면서 무슨 애를 낳으라고.', '일한다고 안주면 누가 일하냐? 이 놈의 지원금은 일하며 꼬박꼬박 세금 내는 사람은 쥐뿔도 없음.', '육아휴직 급여로 애 키우기 힘들어요. \n정부에서 이런 지원마저 해주지않는다면 아이는 더 안낳을거에요. \n월급이외에도 나라가 지원해준다라는 인식이 있어야 애갖는데 부담이 덜해집니다.', '일하는사람은 쎄빠지게 일하고 세금내고\n육휴급여 받으니 못준다 이러고\n집에 있는사람은 육휴 못받으니\n부모급여주겠다\n이게 무슨말인가요?\n진짜 일하는사

### 크롤링 하나로 합치기

In [20]:
import pandas as pd
policy1 = pd.read_csv('/content/영아수당 지급.csv',index_col=0)
policy2 = pd.read_csv('/content/부부육아 휴직 활성화.csv',index_col=0)
policy3 = pd.read_csv('/content/다자녀 지원 확대.csv',index_col=0)
policy4 = pd.read_csv('/content/공보육 확충.csv',index_col=0)
policy5 = pd.read_csv('/content/첫만남 꾸러미 도입.csv',index_col=0)

In [21]:
policy1['policy'] = '영아수당 지급'
policy2['policy'] = '부부육아 휴직 활성화'
policy3['policy'] = '다자녀 지원 확대'
policy4['policy'] = '공보육 확충'
policy5['policy'] = '첫만남 꾸러미 도입'

In [22]:
policy = pd.concat([policy1,policy2])
policy

,reply,policy
0,무슨 중복이요? 그럼 휴직급여 그거 하나 받고 키우라는건디 다 일 그만두라는거에요 ...,영아수당 지급
1,세금 내고 일하는 워킹맘이 죄인인가? 부모급여 줄 것처럼 언플해서 원래 지자체에서 ...,영아수당 지급
2,와 1년 주는거도 건드리네 더줄생각은 못하고,영아수당 지급
3,통상임금 80프로 해봐야 물가대비 애키우는돈에 턱없이부족한데 저거 70받을라고 일다...,영아수당 지급
4,제정신인가 OOO들이 그럼 다실업자되고 70만원받고땐치라는건가?,영아수당 지급
...,...,...
2801,그렇다고 윤석열과 조선일보가\n 대안은 아니다. 일제의 후예\n 한나...,부부육아 휴직 활성화
2802,NaN,부부육아 휴직 활성화
2803,NaN,부부육아 휴직 활성화
2804,https://n.news.naver.com/article/607/000000102...,부부육아 휴직 활성화


In [ ]:
policy = pd.concat([policy,policy5])
policy

In [26]:
policy.to_csv('/content/저출산 정책 기사_댓글 모음.csv')